In [1]:
! pip install -U dlt

In [2]:
!pip install "dlt[duckdb]"

In [4]:
import dlt

dlt.pipeline(destination='duckdb', dataset_name='mydata').run([{'id': 1, 'name': 'John'}], table_name="users")

LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x0000023F9D4C3F90>, destination_name='duckdb', destination_displayable_credentials='duckdb:///C:\\Users\\srinivas kothapalli\\anaconda3\\dlt_ipykernel_launcher.duckdb', staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='mydata', loads_ids=['1698996119.580089'], load_packages=[LoadPackageInfo(load_id='1698996119.580089', package_path='C:\\Users\\srinivas kothapalli\\.dlt\\pipelines\\dlt_ipykernel_launcher\\load\\loaded\\1698996119.580089', state='loaded', schema_name='dlt_ipykernel_launcher', schema_update={}, completed_at=DateTime(2023, 11, 3, 7, 22, 0, 735739, tzinfo=Timezone('UTC')), jobs={'failed_jobs': [], 'started_jobs': [], 'new_jobs': [], 'completed_jobs': [LoadJobInfo(state='completed_jobs', file_path='C:\\Users\\srinivas kothapalli\\.dlt\\pipelines\\dlt_ipykernel_launcher\\load\\loaded\\1698996119.580089\\completed_jobs\\users.f35a649e00.0.insert_values', file_size=160,

In [5]:
pipeline = dlt.pipeline(destination="duckdb", dataset_name="country_data")

data = [
    {'country': 'USA', 'population': 331449281, 'capital': 'Washington, D.C.'},
    {'country': 'Canada', 'population': 38005238, 'capital': 'Ottawa'},
    {'country': 'Germany', 'population': 83019200, 'capital': 'Berlin'}
]

info = pipeline.run(data, table_name="countries")

print(info)

Pipeline dlt_ipykernel_launcher completed in 1.83 seconds
1 load package(s) were loaded to destination duckdb and into dataset country_data
The duckdb destination used duckdb:///C:\Users\srinivas kothapalli\anaconda3\dlt_ipykernel_launcher.duckdb location to store data
Load package 1698996121.235189 is LOADED and contains no failed jobs


In [6]:
import dlt

# have data? dlt likes data
data = [{'id': 1, 'name': 'John'}]

# open connection
pipeline = dlt.pipeline(
    destination='duckdb',
    dataset_name='raw_data'
)

# Upsert/merge: Update old records, insert new
load_info = pipeline.run(
    data,
    write_disposition="merge",
    primary_key="id",
    table_name="users"
)

In [7]:
import os
os.environ['GIT_PYTHON_REFRESH'] = 'quiet'

In [ ]:
C:\Users\srinivas kothapalli\Downloads

In [9]:
#C:\Users\srinivas kothapalli\Desktop\my_project_sadanand
import os
os.environ['GIT_PYTHON_GIT_EXECUTABLE'] = 'C:\Program Files\Git\cmd\git.exe'

In [10]:
import git
git.refresh()

In [11]:
venv = dlt.dbt.get_venv(pipeline)
dbt = dlt.dbt.package(
    pipeline,
    "https://github.com/dbt-labs/jaffle_shop.git",
    venv=venv
)
models_info = dbt.run_all()

# Load metadata for monitoring and load package lineage.
# This allows for both row and column level lineage,
# as it contains schema update info linked to the loaded data
pipeline.run([load_info], table_name="loading_status", write_disposition='append')
pipeline.run([models_info], table_name="transform_status", write_disposition='append')

LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x0000023F9F04F690>, destination_name='duckdb', destination_displayable_credentials='duckdb:///C:\\Users\\srinivas kothapalli\\anaconda3\\dlt_ipykernel_launcher.duckdb', staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='raw_data', loads_ids=['1698996177.20831'], load_packages=[LoadPackageInfo(load_id='1698996177.20831', package_path='C:\\Users\\srinivas kothapalli\\.dlt\\pipelines\\dlt_ipykernel_launcher\\load\\loaded\\1698996177.20831', state='loaded', schema_name='dlt_ipykernel_launcher', schema_update={}, completed_at=DateTime(2023, 11, 3, 7, 22, 58, 308798, tzinfo=Timezone('UTC')), jobs={'failed_jobs': [], 'started_jobs': [], 'new_jobs': [], 'completed_jobs': [LoadJobInfo(state='completed_jobs', file_path='C:\\Users\\srinivas kothapalli\\.dlt\\pipelines\\dlt_ipykernel_launcher\\load\\loaded\\1698996177.20831\\completed_jobs\\transform_status.82076ff2ce.0.insert_values', file

In [12]:
from dlt.common.runtime.slack import send_slack_message

slack_hook = "https://hooks.slack.com/services/xxx/xxx/xxx"

for package in load_info.load_packages:
    for table_name, table in package.schema_update.items():
        for column_name, column in table["columns"].items():
            send_slack_message(
                slack_hook,
                message=f"\tTable updated: {table_name}: Column changed: {column_name}: {column['data_type']}"
            )

In [13]:
@dlt.resource(primary_key="id", write_disposition="merge")
def github_repo_events():
    yield from _get_event_pages()